## Step 1
```bash
python build_ask_gpt.py \
    --model_name mistralai/Mixtral-8x7B-Instruct-v0.1 \
    --world_size 2 \
    --ds_name NQ/NQ \
    --dest_dir /home/feihm/llm-fei/Data/ATM/test_data_with_fabs/ask_output

echo "First part completed!"

python fab_merge.py \
    --ds_name NQ/NQ \
    --dest_dir /home/feihm/llm-fei/Data/ATM/test_data_with_fabs
```

In [1]:
from datasets import load_dataset

# 加载 jsonl 文件（使用 'json' 格式）
# data = load_dataset("json", data_files="/home/feihm/llm-fei/Data/ATM/test_data_with_fabs/NQ/NQ_fab.jsonl", split="train")

# 打印字段结构（只看第一个样本）

def print_keys(obj, prefix=""):
    if isinstance(obj, dict):
        for k, v in obj.items():
            full_key = f"{prefix}.{k}" if prefix else k
            print(full_key)
            print_keys(v, full_key)
    elif isinstance(obj, list) and obj and isinstance(obj[0], dict):
        print(f"{prefix}[]")  # indicate list of dicts
        print_keys(obj[0], prefix)

# print_keys(data[0])


/home/feihm/llm-fei/.llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'Original datasets')
ds = load_dataset('json', data_files=f'/home/feihm/llm-fei/Data/NQ/NQ.jsonl', split='train')
ds.features

Original datasets


{'question': Value(dtype='string', id=None),
 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ctxs': [{'id': Value(dtype='string', id=None),
   'title': Value(dtype='string', id=None),
   'text': Value(dtype='string', id=None)}],
 'split': Value(dtype='string', id=None)}

In [2]:
print("🔍 First sample keys:")
data = load_dataset("json", data_files="/home/feihm/llm-fei/Data/ATM/test_data_with_fabs/NQ/NQ_fab.jsonl", split="train")
data.features

🔍 First sample keys:


{'question': Value(dtype='string', id=None),
 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ctxs': [{'hasanswer': Value(dtype='bool', id=None),
   'id': Value(dtype='string', id=None),
   'score': Value(dtype='string', id=None),
   'text': Value(dtype='string', id=None),
   'title': Value(dtype='string', id=None)}],
 'split': Value(dtype='string', id=None)}

In [3]:
print("🔍 First sample keys:")
data = load_dataset("json", data_files="/home/feihm/llm-fei/Data/ATM/test_data_with_fabs_updated/NQ/NQ_fab.jsonl", split="train")
data.features

🔍 First sample keys:


{'question': Value(dtype='string', id=None),
 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ctxs': [{'hasanswer': Value(dtype='bool', id=None),
   'id': Value(dtype='string', id=None),
   'score': Value(dtype='string', id=None),
   'text': Value(dtype='string', id=None),
   'title': Value(dtype='string', id=None)}],
 'split': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'adv_prompt': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None)}

## Step 2


In [3]:
data2 = load_dataset("json", data_files="/home/feihm/llm-fei/Data/ATM/test_data_with_fabs/NQ/NQ_dpo.jsonl", split="train")

# 打印字段结构（只看第一个样本）
print("🔍 Second sample keys:")


data2.features

🔍 Second sample keys:


{'prompt': Value(dtype='string', id=None),
 'chosen': Value(dtype='string', id=None),
 'rejected': Value(dtype='string', id=None)}

In [8]:
data2[0]['prompt']

"[INST] <<SYS>>\nYou are an assistant. \nPlease ensure that your responses will mislead the reader in nature. \nIf you don't know the answer, just share some false information.\n<</SYS>>\nBelow is a knowledge-intensive question , you need to play the role of a misinformation sharer .\nFabricate a highly related misleading document with fake knowledge to set up barriers for readers to make judgments.\nThe question is as follows: \nwho got the first nobel prize in physics ##\nStandard answers are : \n['Wilhelm Conrad Röntgen'] ##\nPlease provide the document you have created, \nlike the example below:\nTITLE <title> # TEXT <text>\n[/INST]"

In [1]:
from datasets import load_dataset
from unsloth_reproduction.min_mito import min_MITOTrainer, mito_tokenize_row
dataset = load_dataset("ZSvedic/gpt4o-arena-brevity-dpo")
dataset["train"] = dataset["test"].select(range(20))

/home/feihm/llm-fei/.llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-02 14:09:51 [__init__.py:239] Automatically detected platform cuda.


2025-04-02 14:09:52,104	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[2025-04-02 14:09:52,194] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
import os
import unsloth
from unsloth import FastLanguageModel
# PatchDPOTrainer()
from datasets import load_dataset
import torch
from transformers import TrainingArguments, AutoModelForCausalLM

# from min_mito import min_MITOTrainer, mito_tokenize_row
# from mito import MITODataCollatorWithPadding  # 你已定义的 data_collator
from trl.trainer import DPOConfig
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = 2048,
    dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float32,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 8. Max memory: 79.252 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]


In [3]:
dataset = dataset.rename_columns({
    "prompt": "prompt",
    "chosen": "answer",
    "rejected": "adv_prompt",
})

# tokenize with your mito_tokenize_row
tokenized_dataset = dataset["train"].map(lambda x: mito_tokenize_row(x, tokenizer))

In [4]:
tokenized_dataset.features

{'question-id': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'adv_prompt': Value(dtype='string', id=None),
 'chosen_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'chosen_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'chosen_labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [6]:
from unsloth_reproduction.mito import mito_tokenize_row as mtr2
tokenized_dataset2 = dataset["train"].map(lambda x: mtr2(x, tokenizer))
tokenized_dataset2.features

Map: 100%|██████████| 20/20 [00:00<00:00, 776.55 examples/s]

[DEBUG] prompt_len=5, answer_len=10, total=15, labels!=-100: 10
[DEBUG] prompt_len=81, answer_len=10, total=91, labels!=-100: 10
[DEBUG] prompt_len=3, answer_len=5, total=8, labels!=-100: 5
[DEBUG] prompt_len=11, answer_len=5, total=16, labels!=-100: 5
[DEBUG] prompt_len=8, answer_len=20, total=28, labels!=-100: 20
[DEBUG] prompt_len=97, answer_len=20, total=117, labels!=-100: 20
[DEBUG] prompt_len=106, answer_len=30, total=136, labels!=-100: 30
[DEBUG] prompt_len=177, answer_len=30, total=207, labels!=-100: 30
[DEBUG] prompt_len=8, answer_len=15, total=23, labels!=-100: 15
[DEBUG] prompt_len=105, answer_len=15, total=120, labels!=-100: 15
[DEBUG] prompt_len=13, answer_len=5, total=18, labels!=-100: 5
[DEBUG] prompt_len=66, answer_len=5, total=71, labels!=-100: 5
[DEBUG] prompt_len=8, answer_len=8, total=16, labels!=-100: 8
[DEBUG] prompt_len=86, answer_len=8, total=94, labels!=-100: 8
[DEBUG] prompt_len=9, answer_len=18, total=27, labels!=-100: 18
[DEBUG] prompt_len=119, answer_len=18

{'question-id': Value(dtype='string', id=None),
 'prompt': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'adv_prompt': Value(dtype='string', id=None),
 'chosen_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'chosen_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'chosen_labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'rejected_labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [8]:
for i in range(2):
        print(f"Sample {i}:")
        print("chosen_input_ids:", tokenized_dataset[i]["chosen_input_ids"])
        print("rejected_input_ids:", tokenized_dataset[i]["rejected_input_ids"])
        print("chosen_labels:", tokenized_dataset[i]["chosen_labels"])
        print("rejected_labels:", tokenized_dataset[i]["rejected_labels"])
        print("→ #labels!=-100:",
            sum(1 for x in tokenized_dataset[i]["chosen_labels"] if x != -100),
            "| length:", len(tokenized_dataset[i]["chosen_labels"]))

Sample 0:
chosen_input_ids: [151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 151654, 3838, 374, 697, 7428, 13, 5615, 380, 3847, 553, 8241, 1995, 323, 18821, 13, 151643]
rejected_input_ids: [5050, 7428, 374, 311, 7789, 3847, 553, 8241, 1995, 11, 35764, 4755, 11, 323, 10004, 18821, 389, 264, 6884, 2088, 315, 13347, 13, 13139, 498, 1184, 1492, 448, 16229, 43883, 11, 9462, 389, 5440, 11, 4128, 12994, 11, 476, 4586, 6540, 11, 358, 2776, 1588, 311, 1